In [1]:
import mysql.connector
import json
from os import path
import re
import pandas as pd
import numpy as np

In [2]:
def dbConnect():
	cnx = mysql.connector.connect(user='root', password='Root@123',host ='localhost',database='bsd',use_pure=True)
	cursor = cnx.cursor()
	cursor.execute('set global max_allowed_packet=67108864')
	return cnx,cursor

In [3]:
con,cur = dbConnect()

In [4]:
def dbExecute(cursor,sql):
	cursor.execute(sql)
	rows = cursor.fetchall()
	return rows

In [24]:
sql = 'select distinct author_ID, author_name from Authors_SE;'
authors = dbExecute(cur,sql)

In [25]:
def get_author_topics(authors,cur,thresh):
    author_topic = {}
    for auth_id, auth_name in authors[:]:
        sql = "select topic_id, count(*) from DocTopic_20k_3gram_SE_40 where \
        paper_id in ( \
        select distinct paper_ID paper_id from Paper_Author_Affiliations_SE where author_ID = '"+ auth_id+"') \
        and prob>"+thresh+"group by topic_id;"
        # Initialize no of publications in each topic to 0
        topics = (dbExecute(cur,sql))
        #author_topic[auth_id] = [('Topic_'+str(i),0) for i in range(40) if 'Topic_' + str(i) not in dict(topics)]
        author_topic[auth_id] = dict(topics)
    return author_topic

In [15]:
author_topic = get_author_topics(authors,cur,str(0.1))
ntopics = str(40)
with open('author_topic'+ntopics+'.json', 'w') as f:
    json.dump(author_topic, f)
f.close()

InternalError: Unread result found

In [5]:
domain = 'SE'
ntopics = 40
with open('../author_topic_'+domain+'_'+str(ntopics)+'.json', 'r') as f:
    author_topic = json.load(f)

In [19]:
sql = 'select pa.author_ID, count(distinct p.paper_ID) count, count(distinct paper_published_year) active_years, count(distinct paper_venue_ID) venue,a.author_h_index from \
            Paper_Author_Affiliations_'+domain+' pa,Papers_'+domain+' p, Authors_'+domain+' a \
	        where p.paper_ID =pa.paper_ID and a.author_ID = pa.author_ID group by pa.author_ID;'
authors = dbExecute(cur,sql)
authors = pd.DataFrame(authors,columns =['auth_id','total_publications','total_active_year','no_of_paper_venues','h_index'])
authors.head()

,auth_id,total_publications,total_active_year,no_of_paper_venues,h_index
0,0000635D,2,1,1,0.000
1,00007F5E,2,2,2,0.250
2,00007F65,1,1,1,214.381
3,0001570C,22,14,8,NaN
4,0001C195,1,1,1,NaN


In [26]:
auth_id = '000CE250'
x = 1 
k = len(author_topic[auth_id])
home_grown = 0
sno =0
pub_count = int(authors[authors['auth_id'] == auth_id]['total_publications'])
sim = np.load('sim_mat.npy')
print(list(author_topic[auth_id]))
for i,i_count in author_topic[auth_id].items():
    for j in list(author_topic[auth_id])[x:]:
        j_count = author_topic[auth_id][j]
        home_grown += sim[int(re.search('\d+',i).group())][int(re.search('\d+',j).group())]
        d = 1-sim[int(re.search('\d+',i).group())][int(re.search('\d+',j).group())]
        ss = d*i_count*j_count/(pub_count**2)
        sno += ss
        print(i,j,d,home_grown,ss)
    x+=1
print(home_grown,sno)

['Topic_10', 'Topic_12', 'Topic_19', 'Topic_20', 'Topic_22', 'Topic_23']
Topic_10 Topic_12 0.8171616523216378 0.18283834767836218 0.27238721744054595
Topic_10 Topic_19 0.9043301659428622 0.2785081817355 0.30144338864762077
Topic_10 Topic_20 0.8029909952325376 0.4755171865029624 0.26766366507751255
Topic_10 Topic_22 0.676923203776289 0.7985939827266734 0.22564106792542965
Topic_10 Topic_23 0.9464298512967702 0.8521641314299032 0.3154766170989234
Topic_12 Topic_19 0.9677860225696325 0.8843781088602707 0.10753178028551473
Topic_12 Topic_20 0.9630007406220892 0.9213773682381815 0.10700008229134324
Topic_12 Topic_22 0.873705767627152 1.0476716006110296 0.09707841862523911
Topic_12 Topic_23 0.9850497479852678 1.0626218526257618 0.1094499719983631
Topic_19 Topic_20 0.9458167744315551 1.1168050781942067 0.10509075271461724
Topic_19 Topic_22 0.966836164018804 1.1499689141754026 0.10742624044653377
Topic_19 Topic_23 0.990936640989095 1.1590322731863076 0.11010407122101055
Topic_20 Topic_22 0.934

2.4443031365450496

In [19]:
file = '../sterling_no.csv'
if not path.exists(file):
    print("Generating topics ans sterling numbers")
    sterling()
df = pd.read_csv(file)
authors = np.array(df[df['total_topics']>0]['author_id'])
print(len(authors))

#Get graph edges

print('Connection status:',con.is_connected())
sql = "SELECT p1.paper_ID, p1.author_ID a1, p2.author_ID a2 \
    FROM Paper_Author_Affiliations_SE p1 \
    INNER JOIN Paper_Author_Affiliations_SE p2 ON p1.paper_ID = p2.paper_ID where p1.author_ID <> p2.author_ID;"
graph = dbExecute(cur,sql)
graph = pd.DataFrame(graph, columns =['paper_ID', 'auth1', 'auth2']) 
# Remove authors with topics < 1
graph = graph[graph.auth1.isin(authors) & graph.auth2.isin(authors)]

#create edges
graph = graph.groupby(by=['auth1','auth2']).size().reset_index(name='count')
graph['edges'] = list(zip(graph['auth1'],graph['auth2']))
graph[['edges']] = graph['edges'].apply(sorted)
graph[['edges']] = graph['edges'].apply(tuple)

#For undirected graph drop dupilcate edges
undir_graph = graph.drop_duplicates(subset='edges',keep='first')

print(len(graph))

61249
Connection status: True
266896


In [20]:
print(len(undir_graph))

133448


In [22]:
undir_graph[undir_graph['auth1']=='10A7022B']

,auth1,auth2,count,edges
77144,10A7022B,1547BE9B,42,"(10A7022B, 1547BE9B)"
77145,10A7022B,16312CC3,3,"(10A7022B, 16312CC3)"
77146,10A7022B,16E317AA,1,"(10A7022B, 16E317AA)"
77147,10A7022B,1816FCAD,1,"(10A7022B, 1816FCAD)"
77148,10A7022B,18CD1F32,4,"(10A7022B, 18CD1F32)"
77149,10A7022B,1980D790,1,"(10A7022B, 1980D790)"


In [182]:
import pandas as pd
df = pd.DataFrame(d)

In [183]:
df

,s,f
0,45,52
1,4,2


In [181]:
d

[{'s': 45, 'f': 52}, {'s': 4, 'f': 2}]